In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common import NoSuchElementException, TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Estrutura DTO (dataclass)
from dataclasses import dataclass

chrome = webdriver.Chrome()
chrome.implicitly_wait(time_to_wait=10)  # aguardar 10 seg antes de levantar exceção

# Roteiro  
Enquanto houver Nome  
Abrir https://curso-web-scraping.pages.dev/#/desafio/3  
Capturar Nome  
Abrir https://curso-web-scraping.pages.dev/#/desafio/2  
Pesquisar por Nome  
Coletar todos registros  
Voltar https://curso-web-scraping.pages.dev/#/desafio/3  
Cadastrar registros    
Passar Proximo Nome  

Preparar ambiente

In [20]:
# server local
desafio_3 = 'http://localhost:8000/#/desafio/3'
desafio_2 = 'http://localhost:8000/#/desafio/2'

chrome.get(desafio_3)
chrome.switch_to.new_window('tab')
chrome.get(desafio_2)

In [21]:
def get_dicionario_janelas():
    dic_janelas = {}

    chrome.switch_to.window(window_name=chrome.window_handles[0]) # pega ultima janela aberta e torna ativa  # ativa janela

    for aba in chrome.window_handles:
        chrome.switch_to.window(window_name=aba)
        titulo = chrome.title
        print(titulo, ":", aba, "url:", chrome.current_url)
        dic_janelas[titulo] = aba
            
    return dic_janelas

In [22]:
# cria classe com dataclass

@dataclass
class Usuario:
    avatar: str
    nome: str
    profissao: str
    email: str
    telefone: str
    username: str
    estado: str

# lista de Usuarios
lista_usuarios = []

In [23]:
janelas = get_dicionario_janelas()

Desafio 3 : A52156E37A21EDF373F8FAE7BD1BBF16 url: http://localhost:8000/#/desafio/3
Desafio 2 : 8E0FCCE6E21D5B5BD792D2DCA6444506 url: http://localhost:8000/#/desafio/2


In [24]:
def raspa_pagina(nome: str):
    #Abrir https://curso-web-scraping.pages.dev/#/desafio/2  
    chrome.switch_to.window(window_name=janelas['Desafio 2'])


    #mapear elementos
    busca = chrome.find_element(By.CSS_SELECTOR, 'input[type="text"]')
    btn_busca = chrome.find_element(By.CSS_SELECTOR, 'button.text-white')
    lista_usuarios = []

    busca.clear()
    busca.send_keys(nome)
    btn_busca.click()
    # aguardar carregar
    # Configurar Espera de elementos da pagina
    tempo_max = 20  # tempo maximo de espera
    tempo_tentativa = 1  # a cada segundo
    wait = WebDriverWait(driver=chrome, timeout=tempo_max, poll_frequency=tempo_tentativa)
    #aguadar divs de usuario
    div = wait.until(
        EC.presence_of_element_located(locator=(By.CSS_SELECTOR, 'div.users-list')))
    # print(div.get_attribute('class'))

    divs_usuarios = chrome.find_elements(By.CSS_SELECTOR, 'div.users-list > div.items-center')
    for usuario in divs_usuarios:
        usuario = Usuario(
            avatar=usuario.find_element(By.CSS_SELECTOR, 'img').get_attribute('src').strip(),
            nome=usuario.find_element(By.CSS_SELECTOR, 'h3.text-xl').text.strip(),
            profissao=usuario.find_element(By.CSS_SELECTOR, 'span.text-gray-500').text.strip(),
            email=usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(1)').text.split(":")[1].strip(),
            telefone=usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(2)').text.split(":")[1].strip(),
            username=usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(3)').text.split(":")[1].strip(),
            estado=usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(4)').text.split(":")[1].strip()
        )
        lista_usuarios.append(usuario)

    return lista_usuarios

Implementacao do Roteiro

In [25]:
#Enquanto houver Nome  <> Parabens
#Abrir https://curso-web-scraping.pages.dev/#/desafio/3  
chrome.switch_to.window(window_name=janelas['Desafio 3'])
#Capturar Nome
# #usuario  
ele_nome = chrome.find_element(By.ID, 'usuario')
nome = ele_nome.text

lista_usuarios.extend(raspa_pagina(nome=nome))
print('Usuarios na pagina.')
for i, usuario in enumerate(lista_usuarios, start=1):
    print(i, usuario)

#cadastrar usuarios no desafio 2
# cadastrar_usuarios(lista_usuarios)

#Passar Proximo Nome


Usuarios na pagina.
1 Usuario(avatar='https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/451.jpg', nome='Dalton Oliveira', profissao='Especialista', email='dalton.oliveira@example.net', telefone='+55 (60) 5905-1243', username='dalton_oliveira79', estado='Rio Grande do Sul')
2 Usuario(avatar='https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/681.jpg', nome='Dalton Silva', profissao='Orquestrador', email='dalton.silva50@example.com', telefone='+55 (30) 8537-7392', username='dalton.silva', estado='Maranhão')
3 Usuario(avatar='https://avatars.githubusercontent.com/u/23410026', nome='Dalton Franco', profissao='Engenheiro', email='dalton.franco90@example.net', telefone='+55 (75) 1412-8866', username='dalton_franco', estado='Rondônia')
4 Usuario(avatar='https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/601.jpg', nome='Dalton Melo', profissao='Atentende', email='dalton_melo15@example.o

In [26]:
#def cadastrar_usuarios(lista_usuarios: list):
#Voltar https://curso-web-scraping.pages.dev/#/desafio/3
chrome.switch_to.window(window_name=janelas['Desafio 3'])
#Cadastrar registros    
#mapear campos
nome_completo = chrome.find_element(By.NAME, 'nome')
profissao = chrome.find_element(By.NAME, 'profissao')
email = chrome.find_element(By.NAME, 'email')
telefone = chrome.find_element(By.NAME, 'telefone')
username = chrome.find_element(By.NAME, 'usuario')
estado = Select(chrome.find_element(By.NAME, 'estado'))


for usuario in lista_usuarios:
    #limpar campos
    nome_completo.clear()
    profissao.clear()
    email.clear()
    telefone.clear()
    username.clear()

    #preenche campos
    nome_completo.send_keys(usuario.nome)
    profissao.send_keys(usuario.profissao)
    email.send_keys(usuario.email)
    telefone.send_keys(usuario.telefone)
    username.send_keys(usuario.username)
    # select
    username.submit()  # mesmo que clicar no botao
